In [38]:
import wandb
from collections import deque
import torch
import numpy as np


In [39]:
try:
    key = key or input()
except NameError:
    key = input('type your wandb api key: ')

In [40]:
wandb.login(key=key)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [41]:
wandb.init(project="my-awesome-project")

accumulate_reward,▅▃▃▃▃▄▃█▃▂▃▄▅▇▇██▅▇▃▄▅▆▄█▂▃▃▄▂▆▁▄▇▆▅▂▁▄▄
action,▆█▆█▆▁▁▆▆██▆██▁█████▁▁██▁▆█▁▁▃▆▁▁▁▁██▃▁▃
action_values[0],▁▂▃▄▄▃▄▁▂▃▂▁▃▃▄▃▃▄▄▆▃▆▆▆▅▅▇▆▆▃▆▄▇▇▆█▇▆▆▅
action_values[1],▁▄▃▂▅▁▃▁▃▂▁▂▂▂▂▂▃▄▄▅▃▆▆▅▅▅▇▆▆▅█▄▇▇▆█▇▇▆▆
action_values[2],▁▂▁▃█▁▁▂▂▂▁▁▂▂▂▂▃▃▃▄▂▄▄▄▄▄▅▄▄▂▅▂▄▅▄▅▅▄▄▄
action_values[3],▁▂▂▄█▂▂▂▃▃▂▁▄▃▂▃▄▅▄▆▂▂▆▅▄▄▆▅▅▃▅▃▅▆▅▆▆▅▄▃
eps,█▇▅▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▇▆▄▇▅▇▇▅▇▅▃▆▄▄▃▃▂▃▂▁▃▂▁▂▁▂▂▂▂▂▄▁▂▂▂▃▁▂▂
reward,▄▂▇▁▂▂▂▂▁▄▇▁▄█▄▄▁▅▄▄▅▁▅▄█▁▅▁▂▁▁▁▁▄▁█▁▅▁▂
rounds,▆▄▃▃▃▃▃▅▁▁▂▂▃▅▄▆▇▃▅▃▃▄▄▂▅▂▄▃▃▂█▁▃▅▇▃▂▁▂▅
accumulate_reward,2230


In [42]:
from Agent import Agent
from Board import Board

In [43]:
BUFFER_SIZE   = int(1e5)
BATCH_SIZE    = 64
GAMMA         = 0.99 # discount factor
TAU           = 1e-3 # soft update of target parameter
LEARNING_RATE = 5e-4
UPDATE_EVERY  = 32    # how often to update the target
STATE_SIZE    = 16
ACTION_SIZE   = 4   

In [44]:
agent = Agent(STATE_SIZE, ACTION_SIZE, LEARNING_RATE)
env = Board()


In [45]:
wandb.watch(agent.qnetwork_local,log_freq=10)

[]

## Monitoring metrics and parameters

### Metrics for how the model doing and updating
1. q values
1. gradients
1. loss
1. eps
1. 


### Metrics for model's performance
1. largest number of the board
1. accumulate reward
1. number of rounds

In [46]:
def train(agent: Agent, n_episodes, max_time_step, eps_start, eps_end, eps_decay):
    scores = []
    num_rounds = []
    scores_window = deque(maxlen=100)
    eps = eps_start
    for episode in range(n_episodes):
        env.reset()
        state = env.board
        accumulate_reward = 0
        rounds = 0
        for time_step in range(max_time_step):
            action_values = agent.q_value(state, eps)
            action = agent.decide(action_values, eps)
            next_state, reward, done = env.step(action)
            agent.step(state, action, reward, next_state, done)
            """ === this step has finished === """
            wandb.log({'action':action, 'reward': reward, 'eps': eps})
            wandb.log({f'action_values[{i}]':q for i, q in enumerate(action_values.cpu().numpy().flatten()) })
            if agent.time_step % UPDATE_EVERY == 0 and len(agent.memory) > BATCH_SIZE:
                loss = agent.learn_from_experience()
                wandb.log({'loss':loss})
            """ === next iteration === """
            state = next_state
            accumulate_reward += reward
            rounds += 1
            if done:
                wandb.log({'rounds':rounds,'accumulate_reward':accumulate_reward, 'max_number':np.max(state)})
                break
        scores_window.append(accumulate_reward)
        scores.append(accumulate_reward)
        num_rounds.append(rounds)
        eps = max(eps_end, eps-eps_decay)
        if episode % 100 == 0:
            print(episode, np.mean(scores_window))
            torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pt')
    return scores, num_rounds

In [49]:
train(agent, n_episodes=1000, max_time_step=1000, eps_start=1.0, eps_end=0.05, eps_decay=0.01)

c:\Users\user\Desktop\py-practice\machine_learning\codebase---machine-learning\1024\Agent.py:124: UserWarning: Using a target size (torch.Size([64, 64])) that is different to the input size (torch.Size([64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = torch.nn.functional.mse_loss(q_expected, q_targets)


0 1720.0
100 1765.7
200 1759.0
300 1744.6
400 1810.5
500 1807.0
600 1792.5
700 1790.0
800 1713.5
900 1713.3


([1720,
  1750,
  1790,
  2530,
  2210,
  1790,
  3150,
  1230,
  1120,
  1550,
  1680,
  1860,
  1200,
  1530,
  1530,
  1790,
  1680,
  1530,
  1310,
  1170,
  2260,
  1950,
  1850,
  1920,
  1510,
  1580,
  1510,
  1760,
  1880,
  1850,
  1970,
  1800,
  1950,
  2040,
  2250,
  1680,
  2030,
  1620,
  1660,
  2070,
  1530,
  2720,
  1520,
  1620,
  1850,
  1570,
  2020,
  1960,
  1500,
  2080,
  1750,
  1980,
  1340,
  1810,
  2180,
  1730,
  1910,
  1470,
  1630,
  1600,
  1460,
  1100,
  1440,
  1220,
  1630,
  2220,
  1570,
  1200,
  2180,
  1610,
  2340,
  2360,
  1940,
  1920,
  1470,
  1560,
  1820,
  1430,
  1690,
  1990,
  1660,
  1530,
  2110,
  1740,
  1550,
  1630,
  1620,
  2100,
  1800,
  1650,
  1240,
  2270,
  2320,
  1850,
  1790,
  1660,
  1760,
  1790,
  1550,
  1730,
  1760,
  1440,
  1860,
  1900,
  2550,
  2240,
  1460,
  1850,
  1650,
  2140,
  1280,
  1630,
  1810,
  1670,
  2300,
  1740,
  1710,
  1700,
  1830,
  1830,
  1290,
  1840,
  1670,
  1690,
  1870,


In [50]:
agent.q_value(np.array([[0,0,2,0],[0,0,2,0],[0,0,2,0],[0,0,2,0]])).cpu().numpy().flatten()

array([53.012848, 56.405907, 53.92368 , 52.914616], dtype=float32)

In [51]:
agent.act(np.array([[0,0,2,0],[0,0,2,0],[0,0,2,0],[0,0,2,0]]), eps=0)

1